In [11]:
array=[[1,2], [2,3], [4, 5], [5,6], [6,7], [7,8], [8,9]]
array=array[-5:]
print(array)

[[4, 5], [5, 6], [6, 7], [7, 8], [8, 9]]


In [12]:
import os
from pymongo import MongoClient

In [13]:
image_collection_path=r"backend/mongodb_sample_images"

In [14]:
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "img_label_db"
COLLECTION_NAME = "images"

In [15]:
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]
print(f"Connected to MongoDB database: '{DB_NAME}'")

Connected to MongoDB database: 'img_label_db'


In [16]:
inserted_count = 0
skipped_count = 0
supported_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp')

In [17]:
print(f"Starting scan of '{image_collection_path}'...")

for root, dirs, files in os.walk(image_collection_path):
    # Determine the label based on the subfolder name
    # If the root is the main IMAGE_COLLECTION_ROOT, there's no specific label for that level,
    # so we'll look at the immediate subfolder.
    relative_path = os.path.relpath(root, image_collection_path)
    
    # Skip the root directory itself unless it contains images directly (which is unlikely for labels)
    if relative_path == ".":
        # Check if there are images directly in the root for a generic label, or skip.
        # For this script, we assume subfolders define labels.
        print(f"Skipping root directory '{root}' as a label source.")
        continue
        
    # The label is the immediate subfolder name
    label = os.path.basename(root)

    print(f"\nProcessing folder: '{root}' (Label: '{label}')")

    for file in files:
        if file.lower().endswith(supported_extensions):
            image_name = file
            image_full_path = os.path.join(root, file)

            document = {
                "image_name": image_name,
                "image_path": image_full_path,
                "labels": [label], # Label is the subfolder name
                # No 'metadata' or 'capture_date' as per your requirements
            }

            try:
                # Optional: Check if a document with this image_path already exists to avoid duplicates
                # if collection.count_documents({"image_path": image_full_path}) == 0:
                collection.insert_one(document)
                print(f"  Inserted: {image_name} (Label: {label})")
                inserted_count += 1
                # else:
                #     print(f"  Skipped (already exists): {image_name}")
                #     skipped_count += 1
            except Exception as e:
                print(f"  Error inserting {image_name}: {e}")
                skipped_count += 1
        else:
            print(f"  Skipped (unsupported extension): {file}")
            skipped_count += 1

print("\n--- Script Finished ---")
print(f"Total images inserted: {inserted_count}")
print(f"Total files skipped: {skipped_count}")

# Close the MongoDB connection
client.close()
print("MongoDB connection closed.")

Starting scan of 'backend/mongodb_sample_images'...
Skipping root directory 'backend/mongodb_sample_images' as a label source.

Processing folder: 'backend/mongodb_sample_images\food grain analytics' (Label: 'food grain analytics')
  Inserted: department of food and public distribution.jpg (Label: food grain analytics)
  Inserted: grain analyzer market size 2022-2032.png (Label: food grain analytics)
  Inserted: Grain-Analysis-Kit.jpg (Label: food grain analytics)
  Inserted: grains.jpg (Label: food grain analytics)
  Inserted: India Gran Analysis Market by Size, Share and Forecast.jpg (Label: food grain analytics)
  Inserted: nitrogen analyzer equipment.png (Label: food grain analytics)
  Inserted: plastic grain analysis kit.jpg (Label: food grain analytics)

Processing folder: 'backend/mongodb_sample_images\mumbai bombing' (Label: 'mumbai bombing')
  Inserted: candelight vigil.jpg (Label: mumbai bombing)
  Inserted: explosions.jpg (Label: mumbai bombing)
  Inserted: fire at taj.jpg (

In [18]:
client=MongoClient(MONGO_URI)
db=client[DB_NAME]
collection=db[COLLECTION_NAME]

In [19]:
print(client)
print(db)
print(collection)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'img_label_db')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'img_label_db'), 'images')


In [20]:
for img in collection.find():
    print(img)

{'_id': ObjectId('6855256390c0abb6fd581e22'), 'image_name': 'candelight vigil.jpg', 'image_path': 'C:\\\\Users\\\\Lenovo\\\\demo\\\\rag-runner-deployment-main\\\\backend\\\\mongodb_sample_images\\mumbai bombing\\candelight vigil.jpg', 'labels': ['mumbai bombing']}
{'_id': ObjectId('6855256390c0abb6fd581e23'), 'image_name': 'explosions.jpg', 'image_path': 'C:\\\\Users\\\\Lenovo\\\\demo\\\\rag-runner-deployment-main\\\\backend\\\\mongodb_sample_images\\mumbai bombing\\explosions.jpg', 'labels': ['mumbai bombing']}
{'_id': ObjectId('6855256390c0abb6fd581e24'), 'image_name': 'fire at taj.jpg', 'image_path': 'C:\\\\Users\\\\Lenovo\\\\demo\\\\rag-runner-deployment-main\\\\backend\\\\mongodb_sample_images\\mumbai bombing\\fire at taj.jpg', 'labels': ['mumbai bombing']}
{'_id': ObjectId('6855256390c0abb6fd581e25'), 'image_name': 'firefighting efforts.jpg', 'image_path': 'C:\\\\Users\\\\Lenovo\\\\demo\\\\rag-runner-deployment-main\\\\backend\\\\mongodb_sample_images\\mumbai bombing\\firefightin